Use the WTO API to request data from [WTO Time Series Website](https://timeseries.wto.org/). <br>


Info on how to use the API [here](https://apiportal.wto.org/)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import urllib.parse
import requests
import pandas as pd
import json
from datetime import datetime
dataFolder='/content/gdrive/My Drive/LDC/Data/' 

import sys
sys.path.append('/content/gdrive/My Drive/LDC/Code')
import mysettings

In [0]:
# Preferential by detailed HS 6-digit ; HS MFN - Simple average with ad valorem equivalents (AVE) (Percent) 
indicators=['HS_P_0070','HS_A_0015'] ; indicator_names=['Preferential','MFN']
INDICATORS=dict(zip(indicator_names,indicators))
#Reporting Countries
reporting=['036','124','918','352','392','554','578','756','792','840'] ; country_names=['Australia', 'Canada', 'European Union', 'Iceland', 'Japan', 'New Zealand', 'Norway', 'Switzerland', 'Turkey', 'United States']
OECD= dict(zip(country_names,reporting))

In [0]:
def query_wto(importer,indicator,year):
  """ Requests WTO data using the API

  Args: importer (str): 'Name of the importing country'
    indicator (str): 'Name of the indicator'

  Returns:
  Pandas dataFrame

  """
  ###
  # Create the URL for the request
  wto='https://api.wto.org/timeseries/v1/data?'
  indic=INDICATORS[indicator]
  c=importer
  pc='HS6'
  maxrows='1000000'
  mykey=mysettings.primary_key()

  params = urllib.parse.urlencode({
      # Request parameters
      'i': indic, #
      'r': OECD[c],
      'ps': year,
      'pc': pc,
      'max': maxrows,
      'fmt': 'json',
      'mode': 'full',
      'dec': 'default',
      'lang': '1',
      'meta': 'false',
      'off': '0',
      'subscription-key':mykey
  })
  url=''.join([wto,params])

  ###
  # Get the data
  req=requests.get(url)
  json_data=req.json()
  ###
  # To Pandas DF
  try:
    df=pd.json_normalize(json_data["Dataset"])
    keep=['ReportingEconomyCode','ReportingEconomy','PartnerEconomyCode','PartnerEconomy','ProductOrSectorCode','Year','TextValue','Value']
    rename={'ReportingEconomyCode': 'importerCode',
            'ReportingEconomy':'importer',
            'PartnerEconomyCode':'exporterCode',	
            'PartnerEconomy':'exporter',
            'ProductOrSectorCode':'HS6',
            'Year':'year',
            'TextValue':'preference',
            'Value':'tariff_pct'}
    df=df[keep]        
    df.rename(columns=rename,inplace=True)
    return df
  except KeyError:
    pass 
  


In [0]:
#Loop over Importers
for imp in country_names:
  #Loop over indicators
  for ind in indicator_names:
    print(f"Started {imp} for {ind}")
    #Create empty list of dataframes
    DF=[]
    print(datetime.now())
    #Loop over each year
    for year in range(1990,2016):
      #Get data for the given: (importer, indicator,year)
      df = query_wto(importer=imp,indicator=ind,year=year)
      #Append dataframe if dataframe was returned
      if isinstance(df,pd.DataFrame):
        print(f"Appending {year}",end=',')
        DF.append(df)
    #Concat all dataframes for the (importer,year)
    print('Concatenating',end=', ')
    df=pd.concat(DF)
    pathOut=''.join([dataFolder ,imp ,'_' , ind , '.csv'])
    print(f"{len(df)} obs.")
    print(datetime.now())
    df.to_csv(pathOut,index=False)


Started Australia for Preferential
2020-05-12 17:01:19.827273
Appending 1996,Appending 1997,Appending 1998,Appending 1999,Appending 2000,Appending 2001,Appending 2002,Appending 2003,Appending 2004,Appending 2005,Appending 2006,Appending 2007,Appending 2008,Appending 2009,Appending 2010,Appending 2011,Appending 2012,Appending 2013,Appending 2014,Appending 2015,Concatenating, 3775555 obs.
2020-05-12 21:43:36.873141
Started Australia for MFN
2020-05-12 21:44:01.329575
Appending 1996,Appending 1997,Appending 1998,Appending 1999,Appending 2000,Appending 2001,Appending 2002,Appending 2003,Appending 2004,Appending 2005,Appending 2006,Appending 2007,Appending 2008,Appending 2009,Appending 2010,Appending 2011,Appending 2012,Appending 2013,Appending 2014,Appending 2015,Concatenating, 102856 obs.
2020-05-12 21:52:47.824901
Started Canada for Preferential
2020-05-12 21:52:48.532182
Appending 1996,Appending 1997,Appending 1998,Appending 1999,Appending 2000,Appending 2001,Appending 2002,Appending 20